In [6]:
!pip install transformers datasets evaluate --quiet


In [13]:
import os
import evaluate
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments
)

# تعطيل WandB
os.environ["WANDB_MODE"] = "disabled"

# ========== 1. تحميل البيانات وتنظيفها ==========
print(" Loading dataset...")
df = pd.read_csv('/kaggle/input/train-csv/train.csv')  # نستخدم الملف الأصلي

# نأخذ الأعمدة المهمة بس
df_clean = df[['resume_text', 'job_description_text', 'label']].copy()

# تحويل الـ labels من نصوص لأرقام
label_map = {
    "No Fit": 0,
    "Potential Fit": 1,
    "Good Fit": 2
}
df_clean['label'] = df_clean['label'].map(label_map)

# التأكد من عدم وجود قيم فارغة
df_clean = df_clean.dropna()

print(f"✅ Label distribution:")
print(df_clean['label'].value_counts().sort_index())

# تحويل لـ Hugging Face Dataset
dataset = Dataset.from_pandas(df_clean)
print(f"✅ Dataset loaded: {len(dataset)} examples")

# ========== 2. تحميل JobBERT-v3 ==========
print("\n🤖 Loading JobBERT-v3...")
model_name = "TechWolf/JobBERT-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=3,
    ignore_mismatched_sizes=True
)
print("✅ Model loaded successfully!")

# ========== 3. Tokenization ==========
print("\n🔄 Tokenizing dataset...")
def tokenize_function(examples):
    return tokenizer(
        examples['resume_text'],
        examples['job_description_text'],
        truncation=True,
        padding='max_length',
        max_length=256
    )

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=['resume_text', 'job_description_text']
)
print("✅ Tokenization complete!")

# ========== 4. Split Data ==========
train_size = int(0.9 * len(tokenized_dataset))
train_dataset = tokenized_dataset.select(range(train_size))
eval_dataset = tokenized_dataset.select(range(train_size, len(tokenized_dataset)))
print(f"\n📊 Train: {len(train_dataset)} | Eval: {len(eval_dataset)}")

# ========== 5. Metrics ==========
print("\n📈 Loading metrics...")
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    
    return {
        "accuracy": accuracy_metric.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision_metric.compute(predictions=preds, references=labels, average="weighted")["precision"],
        "recall": recall_metric.compute(predictions=preds, references=labels, average="weighted")["recall"],
        "f1": f1_metric.compute(predictions=preds, references=labels, average="weighted")["f1"]
    }

# ========== 6. Training Arguments ==========
try:
    training_args = TrainingArguments(
        output_dir="./jobbert_results",
        eval_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        save_strategy="epoch",
        logging_steps=50,
        report_to="none",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1"
    )
except TypeError:
    training_args = TrainingArguments(
        output_dir="./jobbert_results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        save_strategy="epoch",
        logging_steps=50,
        report_to="none",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1"
    )

# ========== 7. Trainer ==========
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# ========== 8. Training ==========
print("\n🚀 Starting JobBERT-v3 Fine-tuning...")
print("=" * 60)
trainer.train()

# ========== 9. Evaluation ==========
print("\n" + "=" * 60)
print("🎯 Evaluating model...")
eval_result = trainer.evaluate()

print("\n✅ Training completed!")
print("\n📊 Final Results:")
print("=" * 60)
print(f"  Accuracy:  {eval_result['eval_accuracy']:.4f}")
print(f"  Precision: {eval_result['eval_precision']:.4f}")
print(f"  Recall:    {eval_result['eval_recall']:.4f}")
print(f"  F1-Score:  {eval_result['eval_f1']:.4f}")
print(f"  Loss:      {eval_result['eval_loss']:.4f}")
print("=" * 60)

# ========== 10. Save Model ==========
print("\n💾 Saving model...")
model.save_pretrained("./jobbert_finetuned")
tokenizer.save_pretrained("./jobbert_finetuned")
print("✅ Model saved to: ./jobbert_finetuned")

# ========== 11. مقارنة النتائج ==========
print("\n📈 Comparison with previous model:")
print("=" * 60)
print(f"  Previous (MiniLM):    Accuracy={0.6144:.4f}, F1={0.7611:.4f}")
print(f"  Current (JobBERT):    Accuracy={eval_result['eval_accuracy']:.4f}, F1={eval_result['eval_f1']:.4f}")
improvement = (eval_result['eval_f1'] - 0.7611) / 0.7611 * 100
print(f"  Improvement:          {improvement:+.2f}%")
print("=" * 60)

 Loading dataset...
✅ Label distribution:
label
0    3143
1    1556
2    1542
Name: count, dtype: int64
✅ Dataset loaded: 6241 examples

🤖 Loading JobBERT-v3...


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at TechWolf/JobBERT-v3 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded successfully!

🔄 Tokenizing dataset...


Map:   0%|          | 0/6241 [00:00<?, ? examples/s]

✅ Tokenization complete!

📊 Train: 5616 | Eval: 625

📈 Loading metrics...

🚀 Starting JobBERT-v3 Fine-tuning...


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and re


🎯 Evaluating model...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



✅ Training completed!

📊 Final Results:
  Accuracy:  0.6416
  Precision: 1.0000
  Recall:    0.6416
  F1-Score:  0.7817
  Loss:      0.8829

💾 Saving model...
✅ Model saved to: ./jobbert_finetuned

📈 Comparison with previous model:
  Previous (MiniLM):    Accuracy=0.6144, F1=0.7611
  Current (JobBERT):    Accuracy=0.6416, F1=0.7817
  Improvement:          +2.70%


In [15]:
import os
import evaluate
import pandas as pd
import numpy as np
import torch
from torch import nn
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments
)
from sklearn.utils.class_weight import compute_class_weight

# تعطيل WandB
os.environ["WANDB_MODE"] = "disabled"

# ========== 1. تحميل البيانات ==========
print("📥 Loading dataset...")
df = pd.read_csv('/kaggle/input/train-csv/train.csv')  # نستخدم الملف الأصلي

df_clean = df[['resume_text', 'job_description_text', 'label']].copy()

if df_clean['label'].dtype == 'object':
    label_map = {"No Fit": 0, "Potential Fit": 1, "Good Fit": 2}
    df_clean['label'] = df_clean['label'].map(label_map)
else:
    df_clean['label'] = df_clean['label'].astype(int)

df_clean = df_clean.dropna()

print(f"📊 Original label distribution:")
print(df_clean['label'].value_counts().sort_index())

# ========== 2. حساب Class Weights ==========
labels = df_clean['label'].values
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

print(f"\n⚖️ Class weights calculated:")
for i, weight in enumerate(class_weights):
    print(f"  Class {i}: {weight:.4f}")

# ========== 3. Custom Trainer مع Class Weights ==========
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # استخدام weighted cross entropy
        loss_fct = nn.CrossEntropyLoss(weight=class_weights_tensor.to(logits.device))
        loss = loss_fct(logits, labels)
        
        return (loss, outputs) if return_outputs else loss

# ========== 4. تحضير Dataset ==========
dataset = Dataset.from_pandas(df_clean[['resume_text', 'job_description_text', 'label']])

print("\n🤖 Loading JobBERT-v3...")
model_name = "TechWolf/JobBERT-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=3,
    ignore_mismatched_sizes=True
)

# ========== 5. Tokenization ==========
print("\n🔄 Tokenizing...")
def tokenize_function(examples):
    return tokenizer(
        examples['resume_text'],
        examples['job_description_text'],
        truncation=True,
        padding='max_length',
        max_length=256
    )

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=['resume_text', 'job_description_text']
)

# ========== 6. Split Data ==========
train_size = int(0.9 * len(tokenized_dataset))
train_dataset = tokenized_dataset.select(range(train_size))
eval_dataset = tokenized_dataset.select(range(train_size, len(tokenized_dataset)))

print(f"\n📊 Train: {len(train_dataset)} | Eval: {len(eval_dataset)}")

# ========== 7. Metrics ==========
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    
    # نستخدم sklearn مباشرة علشان فيها zero_division
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, average="weighted", zero_division=0),
        "recall": recall_score(labels, preds, average="weighted", zero_division=0),
        "f1": f1_score(labels, preds, average="weighted", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }

# ========== 8. Training Arguments (محسّنة) ==========
try:
    training_args = TrainingArguments(
        output_dir="./jobbert_balanced_results",
        eval_strategy="epoch",
        learning_rate=3e-5,  # زودنا شوية
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=5,  # زودنا الـ epochs
        weight_decay=0.01,
        save_strategy="epoch",
        logging_steps=50,
        report_to="none",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",  # نستخدم macro F1
        warmup_steps=100,  # learning rate warmup
    )
except TypeError:
    training_args = TrainingArguments(
        output_dir="./jobbert_balanced_results",
        evaluation_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=5,
        weight_decay=0.01,
        save_strategy="epoch",
        logging_steps=50,
        report_to="none",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        warmup_steps=100,
    )

# ========== 9. Weighted Trainer ==========
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# ========== 10. Training ==========
print("\n🚀 Starting Balanced Fine-tuning...")
print("=" * 60)
trainer.train()

# ========== 11. Evaluation ==========
print("\n" + "=" * 60)
print("🎯 Final Evaluation...")
eval_result = trainer.evaluate()

# ========== 12. تحليل مفصل للنتائج ==========
print("\n✅ Training completed!")
print("\n📊 Final Results:")
print("=" * 60)
print(f"  Accuracy:        {eval_result['eval_accuracy']:.4f}")
print(f"  Precision:       {eval_result['eval_precision']:.4f}")
print(f"  Recall:          {eval_result['eval_recall']:.4f}")
print(f"  F1 (weighted):   {eval_result['eval_f1']:.4f}")
print(f"  F1 (macro):      {eval_result['eval_f1_macro']:.4f}")
print(f"  Loss:            {eval_result['eval_loss']:.4f}")
print("=" * 60)

# ========== 13. تحليل predictions ==========
print("\n🔍 Analyzing predictions per class...")
predictions = trainer.predict(eval_dataset)
preds = predictions.predictions.argmax(axis=-1)
true_labels = predictions.label_ids

from sklearn.metrics import classification_report, confusion_matrix

print("\n📋 Classification Report:")
print(classification_report(true_labels, preds, 
                          target_names=['No Fit', 'Potential Fit', 'Good Fit'],
                          zero_division=0))

print("\n🎯 Confusion Matrix:")
cm = confusion_matrix(true_labels, preds)
print("          Predicted")
print("           0    1    2")
for i, row in enumerate(cm):
    print(f"Actual {i}: {row}")

# ========== 14. Save Model ==========
print("\n💾 Saving model...")
model.save_pretrained("./jobbert_balanced_finetuned")
tokenizer.save_pretrained("./jobbert_balanced_finetuned")
print("✅ Model saved!")

# ========== 15. مقارنة شاملة ==========
print("\n📈 Complete Comparison:")
print("=" * 60)
print(f"  MiniLM (baseline):   Acc={0.6144:.4f}, F1={0.7611:.4f}")
print(f"  JobBERT (basic):     Acc={0.6416:.4f}, F1={0.7817:.4f}")
print(f"  JobBERT (balanced):  Acc={eval_result['eval_accuracy']:.4f}, F1={eval_result['eval_f1']:.4f}")
print("=" * 60)

📥 Loading dataset...
📊 Original label distribution:
label
0    3143
1    1556
2    1542
Name: count, dtype: int64

⚖️ Class weights calculated:
  Class 0: 0.6619
  Class 1: 1.3370
  Class 2: 1.3491

🤖 Loading JobBERT-v3...


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at TechWolf/JobBERT-v3 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔄 Tokenizing...


Map:   0%|          | 0/6241 [00:00<?, ? examples/s]


📊 Train: 5616 | Eval: 625

🚀 Starting Balanced Fine-tuning...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,F1 Macro
1,0.891500,1.008316,0.590400,1.000000,0.590400,0.742455,0.247485
2,0.745700,0.748240,0.790400,1.000000,0.790400,0.882931,0.294310
3,0.635600,0.761987,0.732800,1.000000,0.732800,0.845799,0.281933
4,0.556400,0.711983,0.731200,1.000000,0.731200,0.844732,0.281577
5,0.439000,0.765438,0.704000,1.000000,0.704000,0.826291,0.275430



🎯 Final Evaluation...



✅ Training completed!

📊 Final Results:
  Accuracy:        0.7904
  Precision:       1.0000
  Recall:          0.7904
  F1 (weighted):   0.8829
  F1 (macro):      0.2943
  Loss:            0.7482

🔍 Analyzing predictions per class...

📋 Classification Report:
               precision    recall  f1-score   support

       No Fit       0.00      0.00      0.00         0
Potential Fit       0.00      0.00      0.00         0
     Good Fit       1.00      0.79      0.88       625

     accuracy                           0.79       625
    macro avg       0.33      0.26      0.29       625
 weighted avg       1.00      0.79      0.88       625


🎯 Confusion Matrix:
          Predicted
           0    1    2
Actual 0: [0 0 0]
Actual 1: [0 0 0]
Actual 2: [ 95  36 494]

💾 Saving model...
✅ Model saved!

📈 Complete Comparison:
  MiniLM (baseline):   Acc=0.6144, F1=0.7611
  JobBERT (basic):     Acc=0.6416, F1=0.7817
  JobBERT (balanced):  Acc=0.7904, F1=0.8829
